In [7]:
import torch
import torchvision
import sys
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image

In [8]:
device = torch.device("cuda")

In [8]:
import cv2
import os
from pathlib import Path
from utils.split_video import split_video

 
# Path to the video file
video_path = Path(r"C:\Users\AORUS\Downloads\Screen_Recording_20241207_104321_REDnote.mp4")
split_video(video_path)



Frames extracted and saved to temp\test


In [9]:
from utils.crop_img import crop_image

crop_image([0,521,1076,1597],172)

In [10]:
from utils.load_model import load_model

predictor = load_model("small")

In [11]:
from constant import *

In [3]:
DEVICE

'cuda'

In [12]:
from constant import SPLIT_OUTPUT
inference_state = predictor.init_state(video_path=SPLIT_OUTPUT)

frame loading (JPEG): 100%|██████████| 172/172 [00:07<00:00, 22.28it/s]


In [13]:
predictor.reset_state(inference_state)

In [14]:
import numpy as np

ann_frame_idx = 0  # the frame index we interact with
ann_obj_id = 1  # give a unique id to each object we interact with (it can be any integers)

# Let's add a positive click at (x, y) = (210, 350) to get started
points = np.array([[578, 373],[659,994]], dtype=np.float32)
# for labels, `1` means positive click and `0` means negative click
labels = np.array([1,1], np.int32)
predictor.add_new_points_or_box(
    inference_state=inference_state,
    frame_idx=ann_frame_idx,
    obj_id=ann_obj_id,
    points=points,
    labels=labels,
)


(0,
 [1],
 tensor([[[[-2.6717, -2.6717, -2.8353,  ..., -3.0941, -3.1764, -3.1764],
           [-2.6717, -2.6717, -2.8353,  ..., -3.0941, -3.1764, -3.1764],
           [-2.8707, -2.8707, -3.0081,  ..., -3.0559, -3.1205, -3.1205],
           ...,
           [-3.7733, -3.7733, -3.7227,  ..., -3.8967, -3.8260, -3.8260],
           [-3.9190, -3.9190, -3.8471,  ..., -3.8155, -3.7289, -3.7289],
           [-3.9190, -3.9190, -3.8471,  ..., -3.8155, -3.7289, -3.7289]]]],
        device='cuda:0'))

In [15]:
from utils.segement import video_segement

video_segements = video_segement(predictor=predictor, inference_state=inference_state)

propagate in video:   0%|          | 0/172 [00:00<?, ?it/s]

propagate in video: 100%|██████████| 172/172 [25:58<00:00,  9.06s/it]


In [16]:
len(video_segements)

172

In [17]:
from utils.show_mask import show_mask

temp = show_mask(frame_stride=20,video_segements=video_segements)

Image loaded successfully. Shape: (1076, 1076, 3)
Image loaded successfully. Shape: (1076, 1076, 3)
Image loaded successfully. Shape: (1076, 1076, 3)
Image loaded successfully. Shape: (1076, 1076, 3)
Image loaded successfully. Shape: (1076, 1076, 3)
Image loaded successfully. Shape: (1076, 1076, 3)
Image loaded successfully. Shape: (1076, 1076, 3)
Image loaded successfully. Shape: (1076, 1076, 3)
Image loaded successfully. Shape: (1076, 1076, 3)


In [18]:
temp[0].shape

(1076, 1076, 3)

In [19]:
from utils.show_img import show_image


for i in temp:
    show_image(i)

In [20]:
from utils.crop_img import crop_mask

crop_mask(video_segments=video_segements, frame_len=172)

Saved 0000_masked.png as PNG
Saved 0001_masked.png as PNG
Saved 0002_masked.png as PNG
Saved 0003_masked.png as PNG
Saved 0004_masked.png as PNG
Saved 0005_masked.png as PNG
Saved 0006_masked.png as PNG
Saved 0007_masked.png as PNG
Saved 0008_masked.png as PNG
Saved 0009_masked.png as PNG
Saved 0010_masked.png as PNG
Saved 0011_masked.png as PNG
Saved 0012_masked.png as PNG
Saved 0013_masked.png as PNG
Saved 0014_masked.png as PNG
Saved 0015_masked.png as PNG
Saved 0016_masked.png as PNG
Saved 0017_masked.png as PNG
Saved 0018_masked.png as PNG
Saved 0019_masked.png as PNG
Saved 0020_masked.png as PNG
Saved 0021_masked.png as PNG
Saved 0022_masked.png as PNG
Saved 0023_masked.png as PNG
Saved 0024_masked.png as PNG
Saved 0025_masked.png as PNG
Saved 0026_masked.png as PNG
Saved 0027_masked.png as PNG
Saved 0028_masked.png as PNG
Saved 0029_masked.png as PNG
Saved 0030_masked.png as PNG
Saved 0031_masked.png as PNG
Saved 0032_masked.png as PNG
Saved 0033_masked.png as PNG
Saved 0034_mas

In [26]:
from utils.to_gif import to_gif

to_gif(71,1/41.34)

GIF saved as 'temp\gif\output.gif'
